## ライブラリインポート

In [ ]:
# numpy , pandas
import numpy as np 
import pandas as pd
# scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# 可視化用ライブラリ
import matplotlib.pyplot as plt
import seaborn as sns

#pandasのカラムが100列まで見れるようにする
pd.set_option('display.max_columns', 100)

In [ ]:
# 学習データの読み込み
train_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv',index_col=0)
# 先頭5行をみてみる。
train_df.head()

## データ前処理

In [ ]:
# 予測用データセットの読み込み
test_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv',index_col=0)
all_df = pd.concat([train_df.drop(columns='SalePrice'),test_df])

In [ ]:
test_df.head()

## 一部の数字が入っている変数を文字列に変換


In [ ]:
num2str_list = ['MSSubClass','YrSold','MoSold']
for column in num2str_list:
    all_df[column] = all_df[column].astype(str)

## 欠損値の処理
数字の変数の欠損は「0」,文字列の変数の欠損は「'None'」で埋める

In [ ]:
all_df.info()

In [ ]:
# 変数の型ごとに欠損値の扱いが異なるため、列ごとに処理
for column in all_df.columns:
    # dtypeがobjectの場合、文字列の変数
    if all_df[column].dtype=='O':
        all_df[column] = all_df[column].fillna('None')
    # dtypeがint , floatの場合、数字の変数
    else:
        all_df[column] = all_df[column].fillna(0)

In [ ]:
all_df.info()

## 特徴量エンジニアリング


In [ ]:
# 建物内の総面積 = 1階の面積 + 2階の面積 + 地下の面積
all_df["TotalSF"] = all_df["1stFlrSF"] + all_df["2ndFlrSF"] + all_df["TotalBsmtSF"]
    
# 一部屋あたりの平均面積 = 建物の総面積 / 部屋数
all_df['AreaPerRoom'] = all_df['TotalSF']/all_df['TotRmsAbvGrd']
    
# 築年数 + 最新リフォーム年 : この値が大きいほど値段が高くなりそう
all_df['YearBuiltPlusRemod']=all_df['YearBuilt']+all_df['YearRemodAdd']
    
# お風呂の総面積
# Full bath : 浴槽、シャワー、洗面台、便器全てが備わったバスルーム
# Half bath : 洗面台、便器が備わった部屋)(シャワールームがある場合もある)
# シャワーがない場合を想定してHalf Bathには0.5の係数をつける
all_df['TotalBathrooms'] = (all_df['FullBath'] + (0.5 * all_df['HalfBath']) + all_df['BsmtFullBath'] + (0.5 * all_df['BsmtHalfBath']))
    
# 合計の屋根付きの玄関の総面積 
# Porch : 屋根付きの玄関 日本風にいうと縁側
#df['TotalPorchSF'] = (df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF'])
    
# プールの有無
all_df['HasPool'] = all_df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
    
# 2階の有無
#df['Has2ndFloor'] = df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
    
# ガレージの有無
#df['HasGarage'] = df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
    
# 地下室の有無
#df['HasBsmt'] = df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
    
# 暖炉の有無
#df['HasFireplace'] = df['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
all_df

## 文字列のカテゴリ変数化(One-Hot-Encoding)

In [ ]:
# One-Hot-Encoding前の列数
len(all_df.columns)

In [ ]:
# pd.get_dummiesを使うとカテゴリ変数できる。
all_df = pd.get_dummies(all_df)

In [ ]:
# One-Hot-Encoding後の列数
len(all_df.columns)

In [ ]:
all_df

## 外れ値の除去

In [ ]:
# 学習データと予測データに分割して元のデータフレームに戻す。
train_df = pd.merge(all_df.iloc[train_df.index[0]:train_df.index[-1]],train_df['SalePrice'],left_index=True,right_index=True)
test_df = all_df.iloc[train_df.index[-1]:]

* 物件の価格が400000ドル以上
* 敷地面積が20000平方メートル以上
* 建築年が1920年より前

In [ ]:
train_df = train_df[(train_df['LotArea']<20000) & (train_df['SalePrice']<400000)& (train_df['YearBuilt']>1920)]

## 住宅価格を対数変換
多くの機械学習アルゴリズムは正規分布のデータを想定しているため、正規分布ではないデータに対して精度が出ない場合が多い 

In [ ]:
# 対数変換前のヒストグラム、歪度、尖度
sns.distplot(train_df['SalePrice'])
print(f"歪度: {round(train_df['SalePrice'].skew(),4)}" )
print(f"尖度: {round(train_df['SalePrice'].kurt(),4)}" )

In [ ]:
# SalePriceLogに対数変換した値を入れる。説明の都合上新たなカラムを作るが、基本的にそのまま代入して良い。
train_df['SalePriceLog'] = np.log(train_df['SalePrice'])

In [ ]:
# 対数変換後のヒストグラム、歪度、尖度
sns.distplot(train_df['SalePriceLog'])
print(f"歪度: {round(train_df['SalePriceLog'].skew(),4)}" )
print(f"尖度: {round(train_df['SalePriceLog'].kurt(),4)}" )

## 学習データの説明変数と目的変数、予測データの説明変数に分割

In [ ]:
# 学習データ、説明変数
train_x = train_df.drop(columns = ['SalePrice','SalePriceLog'])
#train_x = train_df.drop(columns = ['SalePrice'])
#train_x.head()
# 学習データ、目的変数
train_y = train_df['SalePriceLog']
#train_y = train_df['SalePrice']

# 予測データ、説明変数
test_x = test_df

## モデルの作成、学習

## ハイパーパラメータチューニング
交差検証(Grid Search)を用いて、ハイパーパラメータのalphaをチューニング  

In [ ]:
# alphaパラメータのリスト
param_list = [0.001, 0.01, 0.1, 1.0, 10.0,100.0,1000.0] 
    
for cnt,alpha in enumerate(param_list):
    # パラメータを設定したラッソ回帰モデル
    lasso = Lasso(alpha=alpha) 
    # パイプライン生成
    #pipeline = make_pipeline(StandardScaler(), lasso)
        
    # 学習データ内でホールドアウト検証のために分割 テストデータの割合は0.3 seed値を0に固定
    X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=0)
        
    # 学習
    #pipeline.fit(X_train,y_train)
    lasso.fit(X_train,y_train)
        
    # RMSE(平均誤差)を計算
    #train_rmse = np.sqrt(mean_squared_error(y_train, pipeline.predict(X_train)))
    #test_rmse = np.sqrt(mean_squared_error(y_test, pipeline.predict(X_test)))
    
    train_rmse = np.sqrt(mean_squared_error(y_train, lasso.predict(X_train)))
    test_rmse = np.sqrt(mean_squared_error(y_test, lasso.predict(X_test)))
    # ベストパラメータを更新
    if cnt == 0:
        best_score = test_rmse
        best_param = alpha
    elif best_score > test_rmse:
        best_score = test_rmse
        best_param = alpha
    
# ベストパラメータのalphaと、そのときのMSEを出力
print('alpha : ' + str(best_param))
print('test score is : ' +str(round(best_score,4)))    

## ベストパラメータを用いてモデル作成

In [ ]:
# ラッソ回帰モデルにベストパラメータを設定
lasso = Lasso(alpha = best_param)
# パイプラインの作成
#pipeline = make_pipeline(StandardScaler(), lasso)
# 学習
#lasso=pipeline.fit(train_X,train_y)
lasso=lasso.fit(train_x,train_y)

#from sklearn.ensemble import RandomForestRegressor
#forest = RandomForestRegressor(n_estimators=100,random_state=0,max_features=25)
#forest = RandomForestRegressor(n_estimators=100,random_state=0)

#forest =forest.fit(train_x,train_y)

#from sklearn.ensemble import GradientBoostingRegressor
#forest = GradientBoostingRegressor(random_state=0,max_depth=3, learning_rate=0.01)
#forest = forest.fit(train_x,train_y)

## 住宅価格予測

In [ ]:
# 結果を予測
#pred = pipeline.predict(test_X)
pred = lasso.predict(test_x)
#pred = forest.predict(test_x)
#print("{}".format(np.sum(lasso.coef_ != 0)))
#print("{}".format(np.sum(forest.coef_ != 0)))

## 予測結果を指数変換

In [ ]:
# 予測結果のプロット
sns.distplot(pred)
# 歪度と尖度
print(f"歪度: {round(pd.Series(pred).skew(),4)}" )
print(f"尖度: {round(pd.Series(pred).kurt(),4)}" )

In [ ]:
# 指数変換

pred_exp = np.exp(pred)
"""
# 指数変換した予測結果をプロット
sns.distplot(pred_exp)
# 歪度と尖度
print(f"歪度: {round(pd.Series(pred_exp).skew(),4)}" )
print(f"尖度: {round(pd.Series(pred_exp).kurt(),4)}" )
"""

In [ ]:
# 400,000より高い物件は除去
"""
pred_exp_ex_outliars = pred_exp[pred_exp<400000]
# 指数変換した予測結果をプロット
sns.distplot(pred_exp_ex_outliars)
# 歪度と尖度
print(f"歪度: {round(pd.Series(pred_exp_ex_outliars).skew(),4)}" )
print(f"尖度: {round(pd.Series(pred_exp_ex_outliars).kurt(),4)}" )
"""

In [ ]:
# 学種データの住宅価格をプロット
sns.distplot(train_df['SalePrice'])
# 歪度と尖度
print(f"歪度: {round(pd.Series(train_df['SalePrice']).skew(),4)}" )
print(f"尖度: {round(pd.Series(train_df['SalePrice']).kurt(),4)}" )

## 提出用csvを作成
sample_submission.csv

In [ ]:
# sample_submission.csvの読み込み
submission_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
# sample_submission.csvの形式を確認するために先頭五行を見てみる。
submission_df.head()

In [ ]:
# 指数変換した値を代入
submission_df['SalePrice'] = pred_exp

In [ ]:
submission_df.to_csv('submission.csv',index=False)

csvをダウンロードして提出！